In [1]:
pip install netCDF4

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install statsmodels

Note: you may need to restart the kernel to use updated packages.


In [91]:
pip install pyhomogeneity

Note: you may need to restart the kernel to use updated packages.


In [92]:
import netCDF4
import matplotlib.pyplot as plt
import datetime 
import statsmodels as statsmodels
import statistics as stats
import decimal
import os
import numpy as np
import copy
import pyhomogeneity as hg

In [139]:


class ReadData:
    """
    ReadData-- generates the dataset
    
    TODO: currently doesn't use generators, so it can be slow.
    """
    DMI_data = {}
    DMI_list = [] #sorted version of the dictionary
    sortedKeys_DMI = []
    TRMM_data = {}
    TRMM_list = []
    TRMM_acc_list = [] #weekly precepitation
    TRMM_mod7 = 6 #this is the only way to get the data to match up with NINO and DMI
    sortedKeys_TRMM = []
    ElNino_data= {}
    ElNino_list = [[],[],[],[]]
    sortedKeys_Nino = [[],[],[],[]] #currently, we have 4 Nino items
    data_dict = {}
    
    #constants
    begin = datetime.date(1998, 1, 1) #begin of dates we look at. We need to start a bit late so we can get 7 previous days to use for acc
    end = datetime.date(2019, 12, 25) #end of dates we look at
    #note that we have to discard this final point to make everything even
    
    
    
    
    def __init__(self):
        
        #in the init phase, we generated each data set and sort the dictionary keys
        
        self.DMI_data = self.generateDataDMI()
        print("DMI data generated")
        self.sortedKeys_DMI = self.sortKeys(self.DMI_data)   
        for i in self.sortedKeys_DMI:
            self.DMI_list.append(self.DMI_data[i])
        
        self.TRMM_data = self.generateDataTrim()
        self.sortedKeys_TRMM = self.sortKeys(self.TRMM_data)  
        for i in self.sortedKeys_TRMM:
            self.TRMM_list.append(self.TRMM_data[i])
        print("TRMM data generated")
        sum_L = self.accumulateTRMM()
        self.TRMM_acc_list = self.takeModN_TRMM(sum_L) #this creates a list with the summed precipitation fo the previous days
        
        #and we only take days that are mod7 == 0
        #
        
        
        
        self.ElNino_data = self.generateDataElNino()
        for i in range(0,len(self.ElNino_data)):
            self.sortedKeys_Nino[i] = self.sortKeys(self.ElNino_data[i])
            for j in self.sortedKeys_Nino[i]:
                self.ElNino_list[i].append(self.ElNino_data[i][j])
                #print(j)
        print("ELNino data generated")
        
    def accumulateTRMM(self):
        TRMM_list_update = [] #because we have to skip first date, we lose one data point
        v_0 = 0
        v_1 = 0
        for i in range(6, len(self.TRMM_list)):
            v_0 += self.TRMM_list[i][0][0]
            v_1 += self.TRMM_list[i][3][3]
            TRMM_list_update.append(copy.deepcopy(self.TRMM_list[i]) ) #arrays are pass by REFRENCE--need to copy.
            for j in range(0,6):
                TRMM_list_update[i-6] += self.TRMM_list[i-j-1]
        print(v_0)
        print(v_1)
        return TRMM_list_update
    def takeModN_TRMM(self,L):
        TRMM_list_res = []
        for i in range(0,len(L)):
            if(i % 7 == self.TRMM_mod7): #only take number with mod of TRMM_mod7
                TRMM_list_res.append(L[i])
                #look at this sorted key, why are we missing the last key?
            
        return TRMM_list_res
    
        
    def sortKeys(self,data):
        return sorted(data,key = lambda key: self.toDateTime(key )-self.begin)
    def findMinData(self,data):
        v = 0
        for key in data.keys():
            diff = self.toDateTime(key )-self.begin
            if(v == 0 or v> diff):
                v = diff
        print(v)
        return v
        
        
    
    """
    Generate DMI dataset--used for (???)
    """
    def generateDataDMI(self):
        iodfile = "dmi.nc" #were reading the DMI file
        f = netCDF4.Dataset(iodfile, 'r')
        SSTAnp=f.variables['DMI'][:]
        weeknp=f.variables['WEDCEN2'][:]
        SSTA=SSTAnp.tolist() #list of DMI SSTA
        week=weeknp.tolist() 
        f.close()
        DMIdate=[]     #DATE OF DMI 
        for i in range(0, len(week)):    
            days=week[i]
            start=datetime.date(1900, 1, 1)
            delta=datetime.timedelta(days)
            offset=start+delta
            DMIdate.append(offset)

        date_DMI={}
        skip = True
        for j in range(0, len(SSTA)): #skip first data point, so we can meet with TRMM
            if(self.isInTimeRange(DMIdate[j])):
                if(not skip):
                    date_DMI[str(DMIdate[j]).replace("-","")] =  SSTA[j]
                else:
                    skip = False #skip one data point
            else:
                pass
                #this data we will just discard for now
        #print("len",len(date_DMI)) 1146 length, since we skip one data point
        return date_DMI
    
    def generateDataElNino(self):
        ninoindicesfile="elnino indices.txt"
        f=open(ninoindicesfile, "r")
        linelist=list(f.readlines())    
        #### ARRAYS OF FORMAT (DATETIME, DMI)
        date_Ninos=[{},{},{},{}]
        #12,3,34,4 are the ninos we take
        
        search_range = [[19,23],[32,36],[45,49],[58,62]]
        
        for j in range(0,len(search_range)):
            skip = True
            for i in range(4, len(linelist)): #skip the first datapoint
                date_str = self.replaceDateTime(linelist[i][1:10])
                if (self.isInTimeRangeStr(date_str)):
                    if(not skip):
                        date_Ninos[j][date_str] =  float(linelist[i][search_range[j][0]:search_range[j][1]])            
                    else:
                        skip = False
                else:
                    pass
                    #this is the date we discard
            print(len(date_Ninos[j]) )  #is 1146, since we skip one point
        f.close() 

        
        
        return date_Ninos
    """
    Generate TRMM dataset
    """
    def generateDataTrim(self): #this is very slow, potentially increase speed at some point
        TRMMfolder="TRMM1/"
        datetime_prec={} #create a dictionary with the dates
        for filename in os.listdir(TRMMfolder):
            if filename.endswith('nc4'):
                f=netCDF4.Dataset(TRMMfolder + str(filename))
                precp=f.variables['precipitation'][:]
                
                date_str = str(filename[11:19])
                #if(self.isInTimeRangeStr(date_str) and not str(self.toDateTime(date_str)-self.begin)=="0:00:00" and int(str(self.toDateTime(date_str)-self.begin).split(" ")[0]) % 7 == self.TRMM_mod7  ):
                if(self.isInTimeRangeStr(date_str)):    
                    datetime_prec[date_str] =  precp
                else:
                    pass
                f.close()
            else:
                pass
        
        #datetime_prec=np.array(datetime_prec)
        return datetime_prec
    def isInTimeRange(self,time):
        if (time >= self.begin and time <= self.end):
            return True
        return False
    def toDateTime(self,time):
        return datetime.date(int(time[0:4]),int(time[4:6]),int(time[6:8]))
    def isInTimeRangeStr(self,time):
        return self.isInTimeRange(self.toDateTime(time))
        
        
    def replaceDateTime(self,string):
        return str(datetime.datetime.strptime(string, '%d%b%Y')).split(" ")[0].replace("-","")
    
    
        
    
    
    





reader = ReadData()
    









DMI data generated
TRMM data generated
46684.300396738574
22556.258356884122
1146
1146
1146
1146
ELNino data generated


In [124]:
class AnalyzeData:
    "class for anaylzing the data we generated from a reader"
    reader_class = None
    snhtsetDMI = []
    snhtsetNino = []
    snhtsetTRMM = []
    mx_lon = 13
    mx_lat = 11
    def __init__(self,reader):
        self.reader_class = reader
        #self.generateAllSNHT()
    def generateAllSNHT(self):
        self.snhtsetTRMM= [] #the snht set is ordered as a list (lon,lat,?)
        for lon in range(0,self.mx_lon):
            lat_list = []
            for lat in range(0,self.mx_lat):
                #print("trmm",type(self.getDatasetTRMM(lon,lat)[0]))
                lat_list.append(self.SNHTALL(self.getDatasetTRMM(lon,lat)))
                pass
            self.snhtsetTRMM.append(lat_list)
        
        #TODO: continue with taking the snhtset
    def getDatasetTRMM(self,lon,lat):
        #right now, we don't check to ensure the list is correctly sized, add that later
        #i,j means for each item, take the ith item, and the jth item from that list, and form it into a dataset
        list_res = []
        
        for item in self.reader_class.TRMM_acc_list:
            list_res.append(item[lon][lat])
        
        return list_res
    #(Old snht test)
    
    def SNHTALL(self,dataset):   #SNHT FOR time series of dimension 1 x n
        n=len(dataset)
        sd=np.std(dataset)
        mean = stats.mean(dataset)
        
        snhtset=[]
        mx_v = 0
        for y in range(1,n-1): #Loops from Week 1 to week n-1
            summ1=0 
            for i in range(0, y): 
                summ1 += dataset[i]-mean
            z_1= (1.0/(y*sd)) * (summ1)
            summ2=0
            for i in range(y, n):
                summ2 += dataset[i]-mean
            z_2= (1/( (n-y)*sd)) * (summ2)
            v = y * (z_1 **2) + (n-y)*(z_2**2)
            if(v > mx_v):
                mx_v = v
            if(v > 9.95):
                print(v)
                print(y)
            snhtset.append(v)
        
        return (snhtset,mx_v)
    
    def SNHT(self,dataset):
        result = hg.snht_test(data)
        print(result)
        return hg.snht_test(data)

    def Pearlson(self,x, y, lag):      #LAG means that we compare x_t against y_{t-lag} where lag is positive
        x=x[lag:]
        y=y[:-lag]
        x_mean=stats.mean(x)
        y_mean=stats.mean(y)
        top=0
        n=len(x)-lag
        for j in range(0, n):
            top=top+(x[j]-x_mean)*(y[j]-y_mean)
        pearlson= top/(n*stats.stdev(x) * stats.stdev(y))   
        return pearlson

analyze = AnalyzeData(reader) 

#analyze.generateAllSNHT()

In [125]:
analyze.generateAllSNHT()

10.487766362359768
140
11.029033448373958
141
10.927398641110047
142
10.312038283382417
143
10.191467629469804
778
9.958178917355243
779
10.997218142708059
780
10.828836627695726
781
11.320656428918834
782
11.45125650754361
783
11.055016630415002
784
10.989275390288148
785
11.299231499883518
786
11.05273319021199
787
11.145308960065568
788
10.859477830628151
789
10.46719817146993
790
11.264829266699131
791
11.477107032113473
792
11.965815651521797
793
13.063760548999042
794
13.164757402474645
795
13.024869397609438
796
12.86253157415044
797
12.806838133287208
798
12.766533862638507
799
12.655433073081618
800
12.657877346087774
801
12.022225420415909
802
11.942213093170942
803
11.512705536847196
804
10.97075011558417
805
10.641891054437696
806
10.281592692989857
807
11.351515980448367
778
11.080137948670586
779
10.992186227818351
780
11.822037053548557
781
13.070118559680612
782
13.071685948138736
783
12.60595603693575
784
12.890571102332073
785
13.085416521235192
786
12.55253801742817


11.90974697055341
9
11.50934122068074
12
12.29429236720805
14
12.957284836712331
15
11.690231175357846
16
13.213360625918218
17
14.071038877479177
18
17.343734172753113
19
15.65617619211379
20
13.89928692495576
21
13.057711658145285
22
12.265194454777703
23
10.323282698904183
24
11.434444249476133
1135
11.367230117789967
1136
10.285822284679886
1137
12.497057256963622
19
10.542930658769972
20
10.222728341669988
340
10.704447035634104
341
11.14764871377801
342
11.624168421676373
343
11.807459738017466
344
11.869828590122573
345
12.369425277978566
346
12.88618752766955
347
12.476803210566038
348
12.918602709400801
349
12.42340527696444
350
12.363615008852676
351
12.327113895246965
352
12.577889928958518
353
13.033385595347557
354
11.99640189630766
355
11.548262656992295
356
11.058096452482985
357
10.173603382252864
358
13.314443577603283
2
11.548653981666146
19
10.20922286375075
20
9.951852801554567
21
10.22744595349517
1131
12.365801933459812
1
13.457224647585258
2
13.283276208485134
5


13.379990035345118
1
13.247545605864124
2
12.364134962978243
5
10.121527133076329
6
10.327287097899523
14
10.37871759461653
15
10.875018963335565
16
14.934483776865761
17
15.6383810769383
18
16.50626173173583
19
14.765111726655109
20
14.73552421053356
21
14.498307783813
22
14.312757771471402
23
12.218750883698661
24
10.326879824172128
25
19.117544116498483
5
16.517127165073543
6
12.801427414268478
7
12.705904867572851
8
12.75501771353621
9
14.86892300912159
10
16.469277893714477
11
20.838027921888962
12
21.47440482005756
13
20.78475591541748
14
19.54534424368158
15
18.418214757299214
16
16.343719556672323
17
16.127624583792116
18
17.815770764715747
19
16.88742701468578
20
14.457762716040827
21
12.630812113484705
22
10.748792219057743
23
9.95670696105329
1139
10.171126660684093
1140
12.20193261273029
12
11.338160765542442
13
12.346752368889122
14
11.71001420157103
15
10.020274553597408
16
10.550477029311345
17
13.113619701004803
18
13.797131489824974
19
12.9760851082098
20
11.0764856497

16.385314621215887
1
10.171203710863916
1
10.357057305128574
10
14.080163022335503
11
14.345583142705753
12
14.588124105583185
13
20.54329941552847
14
18.468120541001703
15
16.230973403751317
16
14.08687398316422
17
16.302216743952066
18
15.721281065629917
19
14.344107208381137
20
12.237784440227973
21
10.516260384499889
22
10.003650868867364
1133
11.51509581573417
1134
13.598453468893549
1135
12.508341045863316
1136
12.94151114273592
1137
11.83514795374494
1140
15.845507681941967
9
17.24450495967005
10
17.71362306243348
11
18.64177495284473
12
17.98735646618083
13
21.352679091372426
14
19.721188464971647
15
18.017488046561706
16
15.37098572836978
17
20.433974471828673
18
18.999703111078762
19
16.684237303361733
20
14.365627119882591
21
12.988357804095594
22
11.136566404572799
23
10.556754539040748
1133
11.639788188271515
1134
13.576895557208701
1135
11.213871651135221
1136
11.483413465024052
1137
13.55033910741306
1140
10.256105193823403
1141
16.54628063492381
9
14.275030246584226
10


10.236914198038827
1134
10.630363130026234
9
10.282986317295865
15
14.049538596112056
18
13.670023289840943
19
12.020750656085273
20
10.232474109595792
21
10.260748709760241
1133
10.027805821475425
1
13.387310450779635
9
10.255351462090767
10
10.770169520286178
15
12.112065981671494
17
15.96214313022656
18
18.2317436097524
19
16.046126994547077
20
13.864974720669666
21
13.377055963788225
22
11.55624415504738
23
14.150693494226152
18
16.626140340721463
19
14.332068820057163
20
12.350461675569003
21
11.996379750525739
22
10.312423058126768
23
14.25301733025206
18
14.505476304856485
19
12.425088854875971
20
10.85564529716326
21
10.726114771415972
22
11.965616625315967
14
11.340754150773154
15
10.983067828603772
18
10.786446703461333
1131
12.270457637034779
1132
14.236068178611262
1133
13.907087636445961
1134
15.415215068606928
1135
14.150529543050217
1136
15.1838074256355
1137
11.524353303408907
1139
14.485191345010572
1140
13.26695050760058
9
12.500980672599969
10
11.670915634493621
11
1

9.989351392304071
226
10.416431668079063
534
10.143105254091779
535
10.229636079866623
536
10.846008447157725
537
10.751287943304707
538
11.400162253778603
539
11.655428010942183
540
11.39846227099191
541
11.455871841442177
542
11.510956808379941
543
11.705143793548778
544
11.367212874465878
545
11.069667571971753
546
10.71342477155321
547
10.605826477187414
548
10.270127072165394
549
10.516923218252202
559
11.09136557535399
560
11.540088533065235
561
12.062657152911171
562
11.935779435528758
563
11.987397168805684
564
11.895345460474704
565
12.170759725226361
566
12.114322292648104
567
12.038509457931472
568
11.68682557904281
569
11.90879367763079
570
11.641010477621537
571
11.341996534294854
572
11.031239020934155
573
10.97446500063566
574
10.64744930379242
575
10.470696518115135
576
11.052435577229138
577
11.783580407575787
578
11.843516417646102
579
11.565791869664679
580
11.435509542131026
581
11.91081896212928
582
11.56060870541909
583
12.007737834573097
584
12.032053358844706
58

In [61]:
len(reader.TRMM_data['20090313'][0]) #its shaped at (n,13,11). 
print(len(reader.TRMM_acc_list))
print(len(reader.DMI_list))


1146
1147


In [140]:
#we can create a csv file here
def writeLine(lst): #list [n,n2]
    res  = ""
    for i in lst:
        string = ""
        for j in i:
            string += str(j) + ","
        res += string[0:len(string)-1]
    
        res += "]"
    return res
    
def writeToFile():
    with open('Precip.txt', 'w') as f:
        for i in reader.TRMM_acc_list:
            f.write(writeLine(i) + "\n")
            
writeToFile()
